# UNIT 4. Анализ первозок из Анапы в зимний период 2017 г

# Получение данных из БД PostgreSQL запросом средствами Python

In [1]:
import psycopg2.extras
import numpy as np
import psycopg2
import pandas as pd
!pip install psycopg2

In [2]:
def basa3():
    query = '''
WITH t AS
  (--стоимость проданных билетов по рейсам
SELECT flight_id,
       sum(amount) sum_tickets,
       count(ticket_no) count_tickets,
       avg(amount) avg_tickets
   FROM dst_project.ticket_flights
   GROUP BY 1),
     a AS
  (--характеристики самолета по количеству мест
SELECT aircraft_code,
       count(seat_no) seats_all
   FROM dst_project.seats
   GROUP BY 1),
     b AS
  (--расчет длительности полетов в минутах)
SELECT f.flight_id,
       f.scheduled_arrival - f.scheduled_departure AS time_fly
   FROM dst_project.flights f)
SELECT f.flight_id,
       f.flight_no,
       f.scheduled_departure::date,
       date_part('month', f.scheduled_departure) month_,
       f.departure_airport dep_air,
       f.arrival_airport arr_air,
       f.aircraft_code air_code,
       c.model,
       a.seats_all,
       t.sum_tickets::int,
       t.count_tickets::int,
       t.avg_tickets::int,
       date_part('hour', b.time_fly) * 60 + date_part('minute', b.time_fly) time_fly_min,
       t.count_tickets/a.seats_all::float zapoln_air
FROM dst_project.flights f
LEFT JOIN a ON f.aircraft_code = a.aircraft_code
LEFT JOIN t ON t.flight_id=f.flight_id
LEFT JOIN b ON b.flight_id=f.flight_id
LEFT JOIN dst_project.aircrafts c ON f.aircraft_code=c.aircraft_code
WHERE departure_airport = 'AAQ'
  AND (date_trunc('month', scheduled_departure) in ('2017-01-01',
                                                    '2017-02-01',
                                                    '2017-12-01'))
  AND status not in ('Cancelled')
ORDER BY zapoln_air
 

    '''.format()
    conn = psycopg2.connect(
        "dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data


data = pd.DataFrame(basa3())

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   flight_id            127 non-null    int64  
 1   flight_no            127 non-null    object 
 2   scheduled_departure  127 non-null    object 
 3   month_               127 non-null    float64
 4   dep_air              127 non-null    object 
 5   arr_air              127 non-null    object 
 6   air_code             127 non-null    object 
 7   model                127 non-null    object 
 8   seats_all            127 non-null    int64  
 9   sum_tickets          118 non-null    float64
 10  count_tickets        118 non-null    float64
 11  avg_tickets          118 non-null    float64
 12  time_fly_min         127 non-null    float64
 13  zapoln_air           118 non-null    float64
dtypes: float64(6), int64(2), object(6)
memory usage: 14.0+ KB


In [4]:
data.head()

,flight_id,flight_no,scheduled_departure,month_,dep_air,arr_air,air_code,model,seats_all,sum_tickets,count_tickets,avg_tickets,time_fly_min,zapoln_air
0,136642,PG0480,2017-01-30,1.0,AAQ,EGO,SU9,Sukhoi Superjet-100,97,531000.0,64.0,8297.0,50.0,0.659794
1,136807,PG0480,2017-02-23,2.0,AAQ,EGO,SU9,Sukhoi Superjet-100,97,531000.0,68.0,7809.0,50.0,0.701031
2,136360,PG0252,2017-02-05,2.0,AAQ,SVO,733,Boeing 737-300,130,1455400.0,97.0,15004.0,100.0,0.746154
3,136122,PG0252,2017-01-08,1.0,AAQ,SVO,733,Boeing 737-300,130,1431000.0,97.0,14753.0,100.0,0.746154
4,136178,PG0252,2017-01-29,1.0,AAQ,SVO,733,Boeing 737-300,130,1434600.0,99.0,14491.0,100.0,0.761538


# Добавление данных из внешних источников

In [5]:
# определим типы используемых самолетов
data.model.unique()

array(['Sukhoi Superjet-100', 'Boeing 737-300'], dtype=object)

Характеристики этих самолетов дают такие цифры затрат авиационного топлива в полете: Boeing 737-300 тратит 2600 кг/ч, Sukhoi Superjet-100 - 1700 кг/ч. В Анапе заправкой самолетов занимается компания ООО "Базовый топливный оператор". По данным за 2017 год: в январе 17 года 1 тонна топлива стоила 40686р без учета НДС, в феврале - 37714 р без учета НДС. НДС в тот момент составлял 18%.

In [6]:
# посчитаем все регулярные рейсы
data.flight_no.unique()

array(['PG0480', 'PG0252', 'PG0194'], dtype=object)

In [7]:
# и направления
data.arr_air.unique()

array(['EGO', 'SVO', 'NOZ'], dtype=object)

Расшифровываем: SVO - Шереметьево Москва,
EGO - Белгород, 
NOZ - Новосибирск

In [8]:
# посмотрим среднюю загрузку рейсов
df1 = data.groupby(['flight_no'])['zapoln_air'].mean()
df1

flight_no
PG0194         NaN
PG0252    0.870143
PG0480    0.929757
Name: zapoln_air, dtype: float64

Как видно из цифр: средняя заполняемость самолета выше всего на рейсе 480 в Белгород, на втором месте рейс 252 в Москву. Оба эти рейса в целом показывают хорошую заполняемость для зимнего периода. Рейс 194 в Новосибирск не имеет данных. Либо не были куплены билеты, либо база прерывается и просто нет данных.

In [9]:
# посмотрим на даты рейса в Новосибирск
data[data.flight_no == 'PG0194'].sort_values('scheduled_departure').head()

,flight_id,flight_no,scheduled_departure,month_,dep_air,arr_air,air_code,model,seats_all,sum_tickets,count_tickets,avg_tickets,time_fly_min,zapoln_air
124,136546,PG0194,2017-01-03,1.0,AAQ,NOZ,733,Boeing 737-300,130,NaN,NaN,NaN,305.0,NaN
118,136523,PG0194,2017-01-10,1.0,AAQ,NOZ,733,Boeing 737-300,130,NaN,NaN,NaN,305.0,NaN
125,136540,PG0194,2017-01-17,1.0,AAQ,NOZ,733,Boeing 737-300,130,NaN,NaN,NaN,305.0,NaN
121,136560,PG0194,2017-01-24,1.0,AAQ,NOZ,733,Boeing 737-300,130,NaN,NaN,NaN,305.0,NaN
122,136544,PG0194,2017-01-31,1.0,AAQ,NOZ,733,Boeing 737-300,130,NaN,NaN,NaN,305.0,NaN


Этот рейс стоит в расписании с 3 января 2017, но информации о билетах на него нет.

In [10]:
df2 = data[['flight_id', 'flight_no', 'arr_air', 'month_',
            'model', 'sum_tickets', 'time_fly_min', 'zapoln_air']]
df2.head()

,flight_id,flight_no,arr_air,month_,model,sum_tickets,time_fly_min,zapoln_air
0,136642,PG0480,EGO,1.0,Sukhoi Superjet-100,531000.0,50.0,0.659794
1,136807,PG0480,EGO,2.0,Sukhoi Superjet-100,531000.0,50.0,0.701031
2,136360,PG0252,SVO,2.0,Boeing 737-300,1455400.0,100.0,0.746154
3,136122,PG0252,SVO,1.0,Boeing 737-300,1431000.0,100.0,0.746154
4,136178,PG0252,SVO,1.0,Boeing 737-300,1434600.0,100.0,0.761538


In [11]:
df_new1 = pd.DataFrame(
    {'model': ['Boeing 737-300', 'Sukhoi Superjet-100'], 'topl': [2600, 1700]})
df_new1

,model,topl
0,Boeing 737-300,2600
1,Sukhoi Superjet-100,1700


In [12]:
df_new2 = pd.DataFrame({'month_': [1, 2], 'price': [48009, 44503]})
df_new2

,month_,price
0,1,48009
1,2,44503


In [13]:
df_3 = df2.merge(df_new1, on='model', how='left')
df_3 = df_3.merge(df_new2, on='month_', how='left')
df_3.head()

,flight_id,flight_no,arr_air,month_,model,sum_tickets,time_fly_min,zapoln_air,topl,price
0,136642,PG0480,EGO,1.0,Sukhoi Superjet-100,531000.0,50.0,0.659794,1700,48009
1,136807,PG0480,EGO,2.0,Sukhoi Superjet-100,531000.0,50.0,0.701031,1700,44503
2,136360,PG0252,SVO,2.0,Boeing 737-300,1455400.0,100.0,0.746154,2600,44503
3,136122,PG0252,SVO,1.0,Boeing 737-300,1431000.0,100.0,0.746154,2600,48009
4,136178,PG0252,SVO,1.0,Boeing 737-300,1434600.0,100.0,0.761538,2600,48009


In [14]:
df_3 = df_3.fillna(0)

# Анализ данных

In [15]:
# проведем расчеты по рейсам
df_3['zatr_topl'] = round((df_3.topl*df_3.price*df_3.time_fly_min/60)/1000, 0)

In [16]:
df_3['profit'] = df_3.sum_tickets-df_3.zatr_topl
df_3['profit1'] = df_3.sum_tickets*.14-df_3.zatr_topl

In [17]:
df_3.head()

,flight_id,flight_no,arr_air,month_,model,sum_tickets,time_fly_min,zapoln_air,topl,price,zatr_topl,profit,profit1
0,136642,PG0480,EGO,1.0,Sukhoi Superjet-100,531000.0,50.0,0.659794,1700,48009,68013.0,462987.0,6327.0
1,136807,PG0480,EGO,2.0,Sukhoi Superjet-100,531000.0,50.0,0.701031,1700,44503,63046.0,467954.0,11294.0
2,136360,PG0252,SVO,2.0,Boeing 737-300,1455400.0,100.0,0.746154,2600,44503,192846.0,1262554.0,10910.0
3,136122,PG0252,SVO,1.0,Boeing 737-300,1431000.0,100.0,0.746154,2600,48009,208039.0,1222961.0,-7699.0
4,136178,PG0252,SVO,1.0,Boeing 737-300,1434600.0,100.0,0.761538,2600,48009,208039.0,1226561.0,-7195.0


In [18]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127 entries, 0 to 126
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   flight_id     127 non-null    int64  
 1   flight_no     127 non-null    object 
 2   arr_air       127 non-null    object 
 3   month_        127 non-null    float64
 4   model         127 non-null    object 
 5   sum_tickets   127 non-null    float64
 6   time_fly_min  127 non-null    float64
 7   zapoln_air    127 non-null    float64
 8   topl          127 non-null    int64  
 9   price         127 non-null    int64  
 10  zatr_topl     127 non-null    float64
 11  profit        127 non-null    float64
 12  profit1       127 non-null    float64
dtypes: float64(7), int64(3), object(3)
memory usage: 13.9+ KB


In [19]:
# посчитаем рентабельность рег рейса
df_6 = df_3.groupby(['flight_no'])['profit','profit1'].sum().reset_index()
df_6

<ipython-input-19-f75aa20e7485>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_6 = df_3.groupby(['flight_no'])['profit','profit1'].sum().reset_index()


,flight_no,profit,profit1
0,PG0194,-5525319.0,-5525319.0
1,PG0252,85063103.0,1718783.0
2,PG0480,38042009.0,1994507.0


При любом способе расчета рейс PG0194 - убыточный

In [20]:
# Выберем точечно убыточные рейсы flight_id по следующим признакам:
# уровень загрузки меньше или равен 75%, а затраты на топливо
# превышают 14% от стоимости проданных билетов
df_7 = df_3[['flight_id', 'arr_air', 'zapoln_air', 'profit1']
            ][(df_3.zapoln_air <= 0.76) | (df_3['profit1'] < 0)]
df_7

,flight_id,arr_air,zapoln_air,profit1
0,136642,EGO,0.659794,6327.0
1,136807,EGO,0.701031,11294.0
2,136360,SVO,0.746154,10910.0
3,136122,SVO,0.746154,-7699.0
4,136178,SVO,0.761538,-7195.0
6,136464,SVO,0.769231,-12319.0
118,136523,NOZ,0.000000,-634519.0
119,136513,NOZ,0.000000,-588181.0
120,136514,NOZ,0.000000,-588181.0
121,136560,NOZ,0.000000,-634519.0


1. Таким образом, все рейсы в Новосибирск оказались убыточными.
2. Несколько рейсов на других направлениях тоже на грани убыточности, что не сказывается на рентабельности регулярных рейсов в целом.

In [21]:
# добавим даты
df_8 = data[['flight_id', 'scheduled_departure']]
df_7 = df_7.merge(df_8, on='flight_id', how='left')
df_7

,flight_id,arr_air,zapoln_air,profit1,scheduled_departure
0,136642,EGO,0.659794,6327.0,2017-01-30
1,136807,EGO,0.701031,11294.0,2017-02-23
2,136360,SVO,0.746154,10910.0,2017-02-05
3,136122,SVO,0.746154,-7699.0,2017-01-08
4,136178,SVO,0.761538,-7195.0,2017-01-29
5,136464,SVO,0.769231,-12319.0,2017-01-09
6,136523,NOZ,0.000000,-634519.0,2017-01-10
7,136513,NOZ,0.000000,-588181.0,2017-02-28
8,136514,NOZ,0.000000,-588181.0,2017-02-07
9,136560,NOZ,0.000000,-634519.0,2017-01-24


# Выгрузка таблиц в Excel

In [22]:
# делаем выгрузку в excel
df_7.to_excel("final_1.xls")
df_6.to_excel("final_2.xls")
data.to_excel("basa_from_sql.xls")
df_3.to_excel("analis.xls")